### 1. Volatility Risk Model (Price Severity)

**Question it answers**

> *“If the system is stressed, how large could price moves be?”*

This model focuses on **market impact**, not on detecting stress itself.

### What goes into the model
- Recent price behavior (volatility persistence)
- Operational stress indicators
- Weather pressure
- Storage tightness

### What the model produces
- A **distribution of possible price moves**, not a single forecast
- Tail-risk probabilities such as:
  - `P(|price move| > 2%)`
  - `P(|price move| > 3%)`

### Interpretation
- Captures **risk severity**, not direction
- Explicitly models heavy-tailed outcomes (extreme events)

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
from models.features.build_model_frame import ModelFrameConfig, build_model_frame
from models.features.derive_forecast_inputs import derive_forecast_inputs
from models.volatility.vol_risk_model import fit_vol_risk_model
from models.volatility.forecast_vol_risk import forecast_vol_risk

In [ ]:
MONGO_DB = os.getenv("MONGO_DB")
MONGO_URI = os.getenv("MONGO_URI")

### 1. Build Model Frame

In [ ]:
cfg = ModelFrameConfig(
    pipeline="algonquin",
    capacity_collection="ebb_algonquin_capacity",
    notices_collection="ebb_algonquin_notices",
    start="2025-10-01",
)

In [ ]:
df=build_model_frame(cfg, mongo_uri=MONGO_URI, mongo_db=MONGO_DB)

### 2. Fit Volatility Risk Model

In [ ]:
model_vol, idata_vol, scalers_vol = fit_vol_risk_model(df)

### 4. Out-of-Sample Checks

In [ ]:
x = derive_forecast_inputs(df)

In [ ]:
p_samp, p_alert = forecast_vol_risk(model_vol, idata_vol, x=x, scalers=scalers_vol)


### 5. Diagnostics & Sanity Checks¶